In [1]:
import os
import numpy as np

In [2]:
### get current path
from pathlib import Path
path = Path.cwd()
path

PosixPath('/content')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls "drive/MyDrive/1-time-series classification in manufacturing/Datasets/PHM2022"

 dataset				  X_test.npy
 MLP_hfawaz.ipynb			  X_train.npy
'PHM2022 challenge.ipynb'		  y_test.npy
 PHM2022_Data_Challenge_Description.pdf   y_train.npy
 Untitled.ipynb


In [5]:
path = "drive/MyDrive/1-time-series classification in manufacturing/Datasets/PHM2022"

In [6]:

X_train = np.load(os.path.join(path, "X_train.npy"))
X_test = np.load(os.path.join(path, "X_test.npy"))

y_train = np.load(os.path.join(path, "y_train.npy"))
y_test = np.load(os.path.join(path, "y_test.npy"))

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((40338, 749, 3), (40338,), (13447, 749, 3), (13447,))

In [7]:
## Load the dataset
#path_dir = 'g:/My Drive/1-time-series classification in manufacturing/Datasets'
#DSname = 'Etching_dataset'
#path = path_dir + "/" + DSname + "/"

In [8]:
#X_train, y_train = load_from_tsfile(os.path.join(path, "ArticularyWordRecognition_TRAIN.ts"), return_data_type="numpy3d")
#X_test, y_test = load_from_tsfile(os.path.join(path, "ArticularyWordRecognition_TEST.ts"), return_data_type="numpy3d")
#X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [9]:
#X_train = np.swapaxes(X_train, 1,2)
#X_test = np.swapaxes(X_test, 1,2)
#X_train.shape, X_test.shape

In [10]:
nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))
nb_classes

12

In [11]:
import sklearn.preprocessing
# transform the labels from integers to one hot vectors
enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()
y_train.shape, y_test.shape

((40338, 12), (13447, 12))

In [12]:
# save orignal y because later we will use binary
y_true = np.argmax(y_test, axis=1)

if len(X_train.shape) == 2:  # if univariate
    # add a dimension to make it multivariate with one dimension 
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

input_shape = X_train.shape[1:]

input_shape

(749, 3)

In [13]:
def create_directory(directory_path):
    if os.path.exists(directory_path):
        return None
    else:
        try:
            os.makedirs(directory_path)
        except:
            # in case another machine created the path meanwhile !:(
            return None
        return directory_path

In [14]:
output_directory = './out/'
create_directory(output_directory)

In [15]:
!ls "drive/MyDrive/1-time-series classification in manufacturing/Code/dl-tsc"

classifiers  LICENSE  main.py  png  README.md  results	utils


In [16]:
import sys
sys.path.append("drive/MyDrive/1-time-series classification in manufacturing/Code/dl-tsc")
from classifiers import mlp

In [17]:
classifier = mlp.Classifier_MLP(output_directory=output_directory, input_shape=input_shape, nb_classes=nb_classes, verbose=True)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 749, 3)]          0         
                                                                 
 flatten (Flatten)           (None, 2247)              0         
                                                                 
 dropout (Dropout)           (None, 2247)              0         
                                                                 
 dense (Dense)               (None, 500)               1124000   
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                                 
 dropout_2 (Dropout)         (None, 500)               0     

In [18]:
classifier.fit(X_train, y_train, X_test, y_test, y_true)

Epoch 1/100
2522/2522 [==============================] - 12s 4ms/step - loss: 2.1456 - accuracy: 0.3388 - val_loss: 1.7102 - val_accuracy: 0.4307 - lr: 0.0010
Epoch 2/100
2522/2522 [==============================] - 10s 4ms/step - loss: 1.7847 - accuracy: 0.4232 - val_loss: 1.5138 - val_accuracy: 0.4709 - lr: 0.0010
Epoch 3/100
2522/2522 [==============================] - 11s 4ms/step - loss: 1.6286 - accuracy: 0.4535 - val_loss: 1.4009 - val_accuracy: 0.5171 - lr: 0.0010
Epoch 4/100
2522/2522 [==============================] - 10s 4ms/step - loss: 1.5220 - accuracy: 0.4800 - val_loss: 1.3182 - val_accuracy: 0.5601 - lr: 0.0010
Epoch 5/100
2522/2522 [==============================] - 11s 4ms/step - loss: 1.4481 - accuracy: 0.4994 - val_loss: 1.2482 - val_accuracy: 0.6027 - lr: 0.0010
Epoch 6/100
2522/2522 [==============================] - 10s 4ms/step - loss: 1.3878 - accuracy: 0.5195 - val_loss: 1.1931 - val_accuracy: 0.6287 - lr: 0.0010
Epoch 7/100
2522/2522 [=======================

In [19]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import pandas as pd

def calculate_metrics(y_true, y_pred, duration, y_true_val=None, y_pred_val=None):
    res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall', 'duration'])
    res['precision'] = precision_score(y_true, y_pred, average='macro')
    res['accuracy'] = accuracy_score(y_true, y_pred)

    if not y_true_val is None:
        # this is useful when transfer learning is used with cross validation
        res['accuracy_val'] = accuracy_score(y_true_val, y_pred_val)

    res['recall'] = recall_score(y_true, y_pred, average='macro')
    res['duration'] = duration
    return res

In [20]:
classifier.predict(X_test, y_true,X_train,y_train,y_test,return_df_metrics = True)

421/421 [==============================] - 1s 2ms/step


,precision,accuracy,recall,duration
0,0.93287,0.94839,0.94026,0.0
